In [22]:
import json
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sentence_transformers import SentenceTransformer
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer

In [23]:
with open('Hit_Table_3JSON.json', 'r') as file:
  f100 = json.load(file)



In [24]:
# Convert JSON to DataFrame
df = pd.json_normalize(f100)
df['num_hits'] = pd.to_numeric(df['num_hits'], errors='raise') 
df['num_employees'] = pd.to_numeric(df['num_employees'], errors='raise') 
df['revenue'] = pd.to_numeric(df['revenue'], errors='raise') 
df['naics_code'] = pd.to_numeric(df['naics_code'], errors='raise') 
df['industry_code'] = pd.to_numeric(df['industry_code'], errors='raise') 
df['total_ips'] = pd.to_numeric(df['total_ips'], errors='raise') 
df['total_domains'] = pd.to_numeric(df['total_domains'], errors='raise') 
df['is_alerted'] = pd.to_numeric(df['is_alerted'], errors='raise') 

print(df['rawText'].isnull().sum())
df = df.dropna(subset=['rawText'])
#print(df)

152


In [25]:
#print(df)
#time.sleep(10000)
# Initialize the sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings for each text entry
embeddings = model.encode(df['rawText'].tolist())

# Run PCA on the embeddings
pca = PCA(n_components=3)
pca_result = pca.fit_transform(embeddings)

# Add PCA results to the DataFrame
df['PCA1'] = pca_result[:, 0]
df['PCA2'] = pca_result[:, 1]
df['PCA3'] = pca_result[:, 2]

In [26]:

kmeans = KMeans(n_clusters=3, random_state=42)  # Adjust n_clusters as needed
kmeans.fit(pca_result)
labels = kmeans.labels_

# Add the cluster labels to the data for reference
df['Cluster'] = labels

/home/aedan_ounsamone_bluevoyant_com/.local/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



In [27]:
print(df[['entity_domain', 'PCA1', 'PCA2','PCA3','Cluster']])



              entity_domain      PCA1      PCA2      PCA3  Cluster
2                 cisco.com  0.196797  0.002258  0.266633        1
6              marriott.com -0.083457  0.007193 -0.100301        0
37                  kns.com -0.243728 -0.188646  0.000679        0
40        garrettmotion.com -0.218841 -0.177512 -0.036519        0
44               steris.com  0.412139  0.210639  0.052756        1
45             livanova.com -0.177758 -0.343060  0.035458        0
47                  aon.com  0.365697  0.047062 -0.226509        1
52                  fcx.com  0.332965  0.022418 -0.296704        1
57          oceaneering.com -0.133140  0.326932  0.485105        2
63            interface.com -0.121476 -0.339940  0.126511        0
72            crowntech.com  0.108848  0.233175  0.204362        1
73            keytronic.com  0.124418 -0.005997 -0.200665        1
80               kinter.com -0.119399  0.044268 -0.174488        0
82              leggett.com  0.491320  0.045440 -0.106726     

In [29]:
# Create a 3D scatter plot
fig = px.scatter_3d(
    df,
    x='PCA1',
    y='PCA2',
    z='PCA3',
    size='num_hits',  # Optional: use if you want variable marker sizes
    color='Cluster',  # Optional: use if you want variable marker colors
    title='3D Scatter Plot of Company Filing Embeddings',
    labels={'x': 'X Axis', 'y': 'Y Axis', 'z': 'Z Axis'}
)

# Show the plot
fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [ ]:

# Generate sample data
#X, _ = make_blobs(n_samples=300, centers=4, random_state=42)

# Determine the optimal number of clusters using the Elbow method
inertia = []
K = range(1, 11)
for k in K:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(pca_result)
    inertia.append(kmeans.inertia_)

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(K, inertia, 'bx-')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
plt.title('Elbow Method for Optimal Number of Clusters')
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=4, random_state=69)  # Adjust n_clusters as needed
kmeans.fit(pca_result)
labels = kmeans.labels_

# Add the cluster labels to the data for reference
df['Cluster'] = labels
industry_codes = {
    11: "Agriculture, Forestry, Fishing and Hunting",
    21: "Mining, Quarrying, and Oil and Gas Extraction",
    22: "Utilities",
    23: "Construction",
    31: "Manufacturing",
    32: "Manufacturing",
    33: "Manufacturing",
    42: "Wholesale Trade",
    44: "Retail Trade",
    45: "Retail Trade",
    48: "Transportation and Warehousing",
    49: "Transportation and Warehousing",
    51: "Information",
    52: "Finance and Insurance",
    53: "Real Estate and Rental and Leasing",
    54: "Professional, Scientific, and Technical Services",
    55: "Management of Companies and Enterprises",
    56: "Administrative and Support and Waste Management and Remediation Services",
    61: "Educational Services",
    62: "Health Care and Social Assistance",
    71: "Arts, Entertainment, and Recreation",
    72: "Accommodation and Food Services",
    81: "Other Services (except Public Administration)",
    92: "Public Administration"
}
df['Industry'] = df['industry_code'].map(industry_codes)

In [ ]:
# Create a 3D scatter plot
fig = px.scatter_3d(
    df,
    x='PCA1',
    y='PCA2',
    z='PCA3',
    size='num_hits',  # Optional: use if you want variable marker sizes
    color='Cluster',  # Optional: use if you want variable marker colors
    title='3D Scatter Plot of Company Filing Embeddings',
    
    labels={'x': 'X Axis', 'y': 'Y Axis', 'z': 'Z Axis'}
)

# Show the plot
fig.show()

In [ ]:
df[0:5]

In [ ]:
#Number of Industry
pivot_table_industry = df.pivot_table(index='Cluster', columns='Industry', aggfunc='size', fill_value=0)

# Plotting
pivot_table_industry.plot(kind='bar', stacked=True, figsize=(12, 8))

# Add titles and labels
plt.title('Distribution of Industries per Cluster')
plt.xlabel('Cluster')
plt.ylabel('Number of Entries')
plt.legend(title='Industry')

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:

#Number of Hits

pivot_table_num_hits = df.pivot_table(index='Cluster', columns='num_hits', aggfunc='size', fill_value=0)

# Plotting
pivot_table_num_hits.plot(kind='bar', stacked=True, figsize=(12, 8))

# Add titles and labels
plt.title('Distribution of Number of Hits per Cluster')
plt.xlabel('Cluster')
plt.ylabel('Number of Entries')
plt.legend(title='Num Hits')

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:

#Number of Employees

pivot_table_num_employees = df.pivot_table(index='Cluster', columns='num_employees', aggfunc='size', fill_value=0)

# Plotting
pivot_table_num_employees.plot(kind='bar', stacked=True, figsize=(12, 8))

# Add titles and labels
plt.title('Distribution of Number of Employees per Cluster')
plt.xlabel('Cluster')
plt.ylabel('Number of Entries')
plt.legend(title='Number of Employees')

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:

#Revenue

pivot_table_revenue = df.pivot_table(index='Cluster', columns='revenue', aggfunc='size', fill_value=0)

# Plotting
pivot_table_revenue.plot(kind='bar', stacked=True, figsize=(12, 8))

# Add titles and labels
plt.title('Distribution of Number of Hits per Cluster')
plt.xlabel('Cluster')
plt.ylabel('Number of Entries')
plt.legend(title='Revenue')

# Show the plot
plt.tight_layout()
plt.show()